In [17]:
# dependencies and setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# import API key

from config import gKey

In [22]:
# create data frame from CSV file

cities_df = pd.read_csv("Output_Data/city_weather.csv")

In [23]:
cities_df.head(10)

,city,lat,lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,vila,42.0304,-8.1588,66.20,34,0,17.27,PT,1613409437
1,sirjan,29.4520,55.6814,54.18,32,0,2.37,IR,1613409437
2,petropavlovsk-kamchatskiy,53.0452,158.6483,5.00,72,0,8.95,RU,1613409438
3,torbay,47.6666,-52.7314,28.99,64,75,21.85,CA,1613409438
4,saskylakh,71.9167,114.0833,-22.52,86,100,9.64,RU,1613409438
5,butaritari,3.0707,172.7902,79.12,80,17,15.26,KI,1613409439
6,hualmay,-11.0964,-77.6139,77.79,61,2,9.80,PE,1613409439
7,ipixuna,-1.7625,-48.8043,78.80,89,75,6.91,BR,1613409439
8,margate,51.3813,1.3862,48.99,77,94,4.00,GB,1613409318
9,conakry,9.5380,-13.6773,87.80,62,4,9.22,GN,1613409439


In [24]:
# configure gmaps

gmaps.configure(api_key = gKey)

In [25]:
# use lat lng as locations and add humidity as weight

locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [26]:
# add heat layer to map

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight))
fig

ValueError: weights must be of the same length as locations or None

In [27]:
cities_df.columns

Index(['city', 'lat', 'lon', 'Max Temp', 'Humidity', 'Cloudiness', 'Windspeed',
       'Country', 'Date'],
      dtype='object')

In [28]:
#create new data frame fitting my weather criteria - hot, humid, and a little breezy
ideal_df = cities_df.loc[(cities_df["Max Temp"] < 110) & 
                         (cities_df["Max Temp"] > 75) &
                         (cities_df["Humidity"] > 50) & 
                         (cities_df["Cloudiness"] < 10) & 
                         (cities_df["Wind Speed"] > 4) & 
                         (cities_df["Wind Speed"] < 20)]
ideal_df.dropna()
ideal_df

KeyError: 'Wind Speed'

In [29]:
# Store into variable named hotel_df & reset index for readability
hotel_df = ideal_df
hotel_df = hotel_df.reset_index().drop(columns="index")

In [30]:
# add a "hotel name" column to the data frame
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
2,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
4,157,charlestown,1,US,1558378784,64,41.38,-71.64,87.01,12.75,
5,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17,
6,247,puerto escondido,5,MX,1558378824,66,15.86,-97.07,86.00,9.17,
7,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,
8,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50,
9,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41,


In [31]:
# set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gKey,
}

# iterate through 
for i, r in hotel_df.iterrows():
#get lat, lng from data frame    
    lat = r["Lat"]
    lng = r["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request
    names = requests.get(base_url, params=params)
    names = names.json()
    
    #save hotel name to hotels dataframe
    try:
        hotel_df.loc[i, "Hotel Name"] = names["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing info... skipping.")
hotel_df        

Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
2,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
4,157,charlestown,1,US,1558378784,64,41.38,-71.64,87.01,12.75,
5,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17,
6,247,puerto escondido,5,MX,1558378824,66,15.86,-97.07,86.00,9.17,
7,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,
8,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50,
9,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41,


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))